In [ ]:
# convert format

import pandas as pd
import numpy as np
import json

csv_paths = [
    "data/annotations/raw/ZZ_9.csv",
]
for csv_path in csv_paths:
    marker = csv_path.split("/")[-1].split(".")[0]

    df = pd.read_csv(csv_path)
    # modify the column name
    for col in df.columns:
        df = df.rename(columns={col: col.strip().strip("\\n")})

    # replace nan with ""
    df = df.replace(np.nan, "")

    video = None
    video_list = []

    for row in df.iterrows():
        row = row[1]
        if row["视频序号"]:
            if video:
                video_list.append(video)
            video = {}
            video["video_id"] = marker + "_" + str(int(row["视频序号"]))
            video["video_url"] = row["视频_url"]
            video["video_duration"] = row["视频长度（min）"]
            video["video_type"] = row["视频类型"]
            video["qa_list"] = []
        qa = {
            "question": row["问题"],
            "answer": row["答案"],
            "question_type": row["问题类型"],
            "knowledge": row["若涉及通用知识推理，则写出推理步骤（知识点）"],
            "reasoning": row["依据（解题思路）"],
        }
        video["qa_list"].append(qa)

    video_list.append(video)

    print(f"Detected {len(video_list)} videos from {marker}")

    # save the video_list to a json file
    with open(f"data/annotations/raw/{marker}.json", "w") as f:
        json.dump(video_list, f, ensure_ascii=False, indent=4)

In [ ]:
# verify urls

import os
import json

processing_config = json.load(open("configs/processing_config.json"))
log_dir = processing_config["log_dir"]
os.makedirs(log_dir, exist_ok=True)

count = 0
annotaions_paths = [
    "data/annotations/raw/CZ_1.json",
    "data/annotations/raw/CZ_2.json",
    "data/annotations/raw/CZ_3.json",
    "data/annotations/raw/CZ_4.json",
    "data/annotations/raw/CZ_5.json",
    "data/annotations/raw/CZ_6.json",
    "data/annotations/raw/CZ_7.json",
    "data/annotations/raw/ZZ_1.json",
    "data/annotations/raw/ZZ_2.json",
    "data/annotations/raw/ZZ_3.json",
    "data/annotations/raw/ZZ_4.json",
    "data/annotations/raw/ZZ_5.json",
    "data/annotations/raw/ZZ_6.json",
    "data/annotations/raw/ZZ_7.json",
    "data/annotations/raw/ZZ_8.json",
]

for annotaions_path in annotaions_paths:
    marker = annotaions_path.split("/")[-1].split(".")[0]
    with open(annotaions_path, "r") as f:
        data = json.load(f)

    for video in data:
        url = video["video_url"]
        if not url.startswith("https://www.youtube.com/watch?v="):
            count += 1
            with open(
                os.path.join(log_dir, f"annotation_processing_error.log"), "a"
            ) as f:
                f.write(f"Error video url from {marker}: {url}" + "\n")

print(f"Total error video urls: {count}")

In [ ]:
# refine & translate

import json
from tqdm import tqdm

from mmagent.utils.chat_api import generate_messages, parallel_get_response
from mmagent.prompts import prompt_refine_qa_list
from mmagent.utils.general import validate_and_fix_python_list

annotaions_paths = [
    # "data/annotations/raw/CZ_6.json",
    # "data/annotations/raw/CZ_7.json",
    # "data/annotations/raw/ZZ_7.json",
    # "data/annotations/raw/ZZ_8.json",
    "data/annotations/raw/ZZ_9.json",
]

for annotaions_path in tqdm(annotaions_paths):
    marker = annotaions_path.split("/")[-1].split(".")[0]
    with open(annotaions_path, "r") as f:
        data = json.load(f)

    inputs = []

    for video in data:
        qa_list = video["qa_list"]
        qa_list = [
            {
                "question": qa["question"],
                "answer": qa["answer"],
                "reasoning": qa["reasoning"],
            }
            for qa in qa_list
        ]
        input = [
            {
                "type": "text",
                "content": prompt_refine_qa_list.format(qa_list=qa_list),
            }
        ]
        inputs.append(input)

    messages = [generate_messages(input) for input in inputs]
    model = "gpt-4o-2024-11-20"

    responses = parallel_get_response(model, messages, timeout=30)[0]

    for video, response in zip(data, responses):
        translated_qa_list = validate_and_fix_python_list(response)
        for qa, translated_qa in zip(video["qa_list"], translated_qa_list):
            qa["question"] = translated_qa["question"]
            qa["answer"] = translated_qa["answer"]
            qa["reasoning"] = translated_qa["reasoning"]

    with open(f"data/annotations/raw/{marker}_refined.json", "w") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
# refine urls and add save path

import os
import json
from tqdm import tqdm


def parse_url(url):
    if not url.startswith("https://www.youtube.com/"):
        return None
    if "&" in url:
        pruned_url = url.split("&")[0]
    else:
        pruned_url = url
    return pruned_url, pruned_url.split("watch?v=")[1]


annotations_paths = [
    "data/annotations/raw/CZ_6_refined.json",
    "data/annotations/raw/CZ_7_refined.json",
    "data/annotations/raw/ZZ_7_refined.json",
    "data/annotations/raw/ZZ_8_refined.json",
    "data/annotations/raw/ZZ_9_refined.json",
]
base_save_dir = "/mnt/hdfs/foundation/longlin.kylin/mmagent/data/raw_videos"

error_count = 0

for annotations_path in annotations_paths:
    marker = annotations_path.split("/")[-1].split(".")[0].strip("_refined")
    target_dir = marker + "/"
    save_dir = os.path.join(base_save_dir, marker)
    os.makedirs(save_dir, exist_ok=True)
    with open(annotations_path, "r") as f:
        videos = json.load(f)

    for video in tqdm(videos):
        url = video["video_url"]
        if not url.startswith("https://www.youtube.com/watch?v="):
            print(url)
            video["path"] = ""
            error_count += 1
            continue
        pruned_url, video_id = parse_url(url)
        save_file = os.path.join(save_dir, video_id + ".mp4")
        video["path"] = save_file
        video["video_url"] = pruned_url

    with open(annotations_path, "w") as f:
        json.dump(videos, f, indent=4, ensure_ascii=False)

print(error_count)

In [ ]:
import json
import os

small_test = []
samples_per_file = 50
with open("data/annotations/raw/CZ_1_refined.json", "r") as f:
    CZ = json.load(f)
with open("data/annotations/raw/ZZ_1_refined.json", "r") as f:
    ZZ = json.load(f)

count = 0
for video in CZ:
    if not video["video_url"].startswith("https://www.youtube.com/watch?v="):
        continue
    if os.path.exists(video["path"]):
        small_test.append(video)
        count += 1
        if count >= samples_per_file:
            break

count = 0
for video in ZZ:
    if not video["video_url"].startswith("https://www.youtube.com/watch?v="):
        continue
    if os.path.exists(video["path"]):
        small_test.append(video)
        count += 1
        if count >= samples_per_file:
            break

with open("data/annotations/small_test.json", "w") as f:
    json.dump(small_test, f, indent=4, ensure_ascii=False)

In [ ]:
import json

data_list = [
    "data/annotations/raw/CZ_1_refined.json",
    "data/annotations/raw/CZ_2_refined.json",
    "data/annotations/raw/CZ_3_refined.json",
    "data/annotations/raw/ZZ_1_refined.json",
    "data/annotations/raw/ZZ_2_refined.json",
    "data/annotations/raw/ZZ_3_refined.json",
    "data/annotations/raw/ZZ_4_refined.json",
]

test_id = []
with open("data/annotations/small_test.jsonl", "r") as f:
    for line in f:
        sample = json.loads(line)
        test_id.append(sample["video_id"])

test_id = list(set(test_id))
print(len(test_id))

# test_set = json.load(open("data/annotations/small_test.json", "r"))
# test_id = [video["video_id"] for video in test_set]


small_training_set = []
for data in data_list:
    with open(data, "r") as f:
        data = json.load(f)
    for video in data:
        if video["video_id"] not in test_id:
            small_training_set.append(video)

print(len(test_id))
print(len(small_training_set))
with open("data/annotations/small_train.json", "w") as f:
    json.dump(small_training_set, f)

In [ ]:
import json
from mmagent.utils.video_processing import get_video_info

def parse_url(url):
    if not url.startswith("https://www.youtube.com/"):
        return None
    if "&" in url:
        pruned_url = url.split("&")[0]
    else:
        pruned_url = url
    return pruned_url, pruned_url.split("watch?v=")[1]

with open("data/annotations/small_train.json", "r") as f:
    videos = json.load(f)

filtered_videos = {}

for video in videos:
    try:
        _, video_id = parse_url(video["video_url"])
        if video_id not in filtered_videos:
            video_info = get_video_info(video["path"])
            if video_info["height"] == 720:
                filtered_videos[video_id] = video
        else:
            filtered_videos[video_id]["qa_list"].extend(video["qa_list"])
    except Exception as e:
        continue

print(len(filtered_videos.items()))

outputs = []

for video_id, video in filtered_videos.items():
    outputs.append(video)

with open("data/annotations/train_500.json", "w") as f:
    json.dump(outputs[:500], f)

In [ ]:
# add paths

import json
from mmagent.utils.general import *

processing_config = json.load(open("configs/processing_config.json"))
save_dir = "/mnt/hdfs/foundation/longlin.kylin/mmagent/data/mems"
clip_dir = "/mnt/hdfs/foundation/longlin.kylin/mmagent/data/video_clips"


def add_paths(data):
    with open(data, "r") as f:
        videos = json.load(f)
    for video in videos:
        video_id = video["path"].split("/")[-1].split(".")[0]
        marker = video["video_id"][:4]
        save_path = os.path.join(
            save_dir, marker, generate_file_name(video["path"]) + ".pkl"
        )
        clip_path = os.path.join(clip_dir, marker, video_id)
        video["mem_path"] = save_path
        video["clip_path"] = clip_path
    with open(data, "w") as f:
        json.dump(videos, f, indent=4, ensure_ascii=False)


# add_paths("data/annotations/small_test.json")
# add_paths("data/annotations/small_train.json")
# add_paths("data/annotations/train_500.json")
# add_paths("data/annotations/full_test.json")
add_paths("data/annotations/all.json")

In [ ]:
import json


def convert_to_jsonl(data):
    with open(data, "r") as f:
        videos = json.load(f)
    qas = []
    for video in videos:
        for qa in video["qa_list"]:
            qas.append(
                {
                    "video_id": video["video_id"],
                    "video_url": video["video_url"],
                    "video_path": video["path"],
                    "clip_path": video["clip_path"],
                    "mem_path": video["mem_path"],
                    "question": qa["question"],
                    "answer": qa["answer"],
                    "reasoning": qa["reasoning"],
                }
            )
    with open(f"data/annotations/{data.split('/')[-1].split('.')[0]}.jsonl", "w") as f:
        for qa in qas:
            f.write(json.dumps(qa) + "\n")


# convert_to_jsonl("data/annotations/small_test.json")
# convert_to_jsonl("data/annotations/small_train.json")
convert_to_jsonl("data/annotations/full_test.json")
# convert_to_jsonl("data/annotations/train_500.json")
# convert_to_jsonl("data/annotations/all.json")

In [ ]:
def check_if_contains_mem(data):
    count = 0
    with open(data, "r") as f:
        videos = json.load(f)
    for video in videos:
        if video["mem_path"]:
            count += 1
    return count


# print(check_if_contains_mem("data/annotations/small_train.json"))
# print(check_if_contains_mem("data/annotations/small_test.json"))
print(check_if_contains_mem("data/annotations/train_500.json"))

In [ ]:
import json

filtered_questions = []

with open(
    "/mnt/bn/videonasi18n/longlin.kylin/mmagent/data/annotations/results/0416/baseline_blindly_answers_verified.jsonl",
    "r",
) as f:
    for i, line in enumerate(f):
        sample = json.loads(line)
        if sample["verify_result"].lower().startswith("yes"):
            filtered_questions.append(i)

with open(
    "/mnt/bn/videonasi18n/longlin.kylin/mmagent/data/annotations/results/0417/verified_small_test.json",
    "r",
) as f:
    samples = json.load(f)
    for i, sample in enumerate(samples):
        if sample["verify_result"].lower().startswith("yes"):
            filtered_questions.append(i)

filtered_questions = list(set(filtered_questions))

print(len(filtered_questions))

data_with_flags = []
with open("data/annotations/small_test.jsonl", "r") as f:
    for i, line in enumerate(f):
        sample = json.loads(line)
        if i in filtered_questions:
            sample["flag"] = True
        else:
            sample["flag"] = False
        data_with_flags.append(sample)

with open("data/annotations/small_test.jsonl", "w") as f:
    for sample in data_with_flags:
        f.write(json.dumps(sample) + "\n")

In [ ]:
# do stat

import json
from tqdm import tqdm


def parse_url(url):
    if not url.startswith("https://www.youtube.com/"):
        return None
    if "&" in url:
        pruned_url = url.split("&")[0]
    else:
        pruned_url = url
    return pruned_url, pruned_url.split("watch?v=")[1]


annotations_paths = [
    "data/annotations/raw/CZ_1.json",
    "data/annotations/raw/CZ_2.json",
    "data/annotations/raw/CZ_3.json",
    "data/annotations/raw/CZ_4.json",
    "data/annotations/raw/CZ_5.json",
    "data/annotations/raw/CZ_6.json",
    "data/annotations/raw/CZ_7.json",
    "data/annotations/raw/ZZ_1.json",
    "data/annotations/raw/ZZ_2.json",
    "data/annotations/raw/ZZ_3.json",
    "data/annotations/raw/ZZ_4.json",
    "data/annotations/raw/ZZ_5.json",
    "data/annotations/raw/ZZ_6.json",
    "data/annotations/raw/ZZ_7.json",
    "data/annotations/raw/ZZ_8.json",
]

all_videos = []

for annotations_path in annotations_paths:
    with open(annotations_path, "r") as f:
        videos = json.load(f)

    for video in tqdm(videos):
        url = video["video_url"]
        pruned_url, video_id = parse_url(url)
        all_videos.append(video_id)

print(len(list(set(all_videos))))

In [ ]:
import json

def parse_url(url):
    if not url.startswith("https://www.youtube.com/"):
        return None
    if "&" in url:
        pruned_url = url.split("&")[0]
    else:
        pruned_url = url
    return pruned_url, pruned_url.split("watch?v=")[1]

training_videos = []

with open("data/annotations/train_500.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        _, video_id = parse_url(data["video_url"])
        if video_id not in training_videos:
            training_videos.append(video_id)

print(len(training_videos))

In [ ]:
import json
from mmagent.utils.video_processing import get_video_info

def parse_url(url):
    if not url.startswith("https://www.youtube.com/"):
        return None
    if "&" in url:
        pruned_url = url.split("&")[0]
    else:
        pruned_url = url
    return pruned_url, pruned_url.split("watch?v=")[1]

training_videos = []

with open("data/annotations/train_500.json", "r") as f:
    training_samples = json.load(f)

for sample in training_samples:
    url = sample["video_url"]
    pruned_url, video_id = parse_url(url)
    if video_id not in training_videos:
        training_videos.append(video_id)
    else:
        print(video_id)

print(len(training_videos))

with open("data/annotations/all.json", "r") as f:
    all_samples = json.load(f)

outputs = {}
count = 0

for sample in all_samples:
    url = sample["video_url"]
    pruned_url, video_id = parse_url(url)
    if video_id not in training_videos:
        if video_id not in outputs:
            video_info = get_video_info(sample["path"])
            if video_info["height"] == 720 or video_info["width"] == 1280:
                outputs[video_id] = sample
        else:
            count += 1
            outputs[video_id]["qa_list"].extend(sample["qa_list"])

with open("data/annotations/full_test.json", "w") as f:
    json.dump(list(outputs.values()), f, indent=4)

print(count)
print(len(outputs))

In [ ]:
import json

annotations_paths = [
    "data/annotations/raw/CZ_1_refined.json",
    "data/annotations/raw/CZ_2_refined.json",
    "data/annotations/raw/CZ_3_refined.json",
    "data/annotations/raw/CZ_4_refined.json",
    "data/annotations/raw/CZ_5_refined.json",
    "data/annotations/raw/CZ_6_refined.json",
    "data/annotations/raw/CZ_7_refined.json",
    "data/annotations/raw/ZZ_1_refined.json",
    "data/annotations/raw/ZZ_2_refined.json",
    "data/annotations/raw/ZZ_3_refined.json",
    "data/annotations/raw/ZZ_4_refined.json",
    "data/annotations/raw/ZZ_5_refined.json",
    "data/annotations/raw/ZZ_6_refined.json",
    "data/annotations/raw/ZZ_7_refined.json",
    "data/annotations/raw/ZZ_8_refined.json",
    "data/annotations/raw/ZZ_9_refined.json",
]

all_annotations = []

for path in annotations_paths:
    with open(path, "r") as f:
        annotations = json.load(f)
    all_annotations.extend(annotations)

with open("data/annotations/all.json", "w") as f:
    json.dump(all_annotations, f)

print(len(all_annotations))